## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Необходимо выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. В моём распоряжении — статистика о платёжеспособности клиентов.



### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [138]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<h5> Вывод </h5>
Всего в таблице 12 столбцов, тип данных у каждого столбца - соответствует своему назначению, кроме days_employed и total_income<br>
Подробно разберём, какие в data столбцы и какую информацию они содержат:
<ul>
 <li>children — количество детей в семье</li>
 <li>days_employed — трудовой стаж в днях</li>
 <li>dob_years — возраст клиента в годах</li>
 <li>dob_years — возраст клиента в годах</li>
 <li>education — образование клиента</li>
 <li>education_id — идентификатор образования</li>
 <li>family_status — семейное положение</li>
 <li>family_status_id — идентификатор семейного положения</li>
 <li>gender — пол клиента</li>
 <li>income_type — тип занятости</li>
 <li>debt — имел ли задолженность по возврату кредитов</li>
 <li>total_income — доход в месяц</li>
 <li>purpose — цель получения кредита</li>
 </ul>

Количество значений в столбцах различается. Это говорит о том, что в данных есть <strong>пропущенные</strong> значения.

### Вывод

Есть несколько проблем, которые нужно решать: пропуски, дубликаты и неправильные типы данных.

### Шаг 2. Предобработка данных

### Обработка пропусков

Получим первые пять строк таблицы для визуального представления данных

In [139]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Воспользуемся функциями для поиска пропущенных значений

In [140]:
data.isnull().sum()
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружились пропущенные значения в столбцах days_employed и total_income. 

Это может быть связано с тем, что многие люди не хотят раскрывать свой доход и стаж работы. Также может быть техническая ошибка в выгрузке данных. Если человек не указал одно из этих значений (стаж коррелирует с доходом), то второе автоматически пропускается. То есть <strong>причиной пропусков может быть как человеческий фактор, так и техническая ошибка</strong>.


<h5>Заполнение пропусков</h5>
Доля пропущенных значений составляет примерно 10% от общего количества. Полностью удалять пропуски нельзя, так как доход напрямую влияет на возможность погашения кредита.Соответственно, пропуски необходимо заполнять.
Так как данные в столбце стаж имеют отрицательные значения и выпадающие значения (стаж 1000 лет), то пропуски логично заполнить значением медианы.  

In [141]:
income_median = data.groupby('income_type')['total_income'].median() #посчитаем значение медианы для каждого типа занятости
print(income_median)

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [143]:
#Заменяем пропущенные значения в столбце total_income на медиану значений этого столбца.
total_income_avg = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value = total_income_avg)



In [144]:
 #Заменяем пропущенные значения в столбце days_employed
data['days_employed'] = abs(data['days_employed']) #меняем отрицательные значения на положительные
days_employed_avg = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(value = days_employed_avg) #заменяем пропущенные значения медианой


In [145]:
#проверяем остались ли какие-либо пропуски
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Количество пропущенных значений составляло приблизительно 10% от всех элементов. Были заменены имеющиеся пропуски в столбцах days_employed и total_income на медианное значение всех элементов столбцов. 

### Замена типа данных

Еще раз выведем первые пять строк таблицы

In [146]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Количество отработанных дней не может быть дробным числом, так как стаж всегда исчисляется в полных днях на текущий момент. Заработная плата тоже исчисляется в целых числах, так как даже после вычета всех налогов человек получает зарплату без начисления копеек. В столбцах days_employed и total_income тип данных float64. Так как количество отработанных дней (стаж) и заработная плата (доход) являются целыми числами, то необходимо заменить тип этих данных на целочиссленный.

In [147]:
data['days_employed'] = data['days_employed'].astype('int') #меняем тип данных столбца days_employed
data['total_income'] = data['total_income'].astype('int') #меняем тип данных столбца total_income
data.info() #проверяем тип данных изменяемых столбцов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [148]:
data.head() #выводим результат

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


В столбце education образование написано по разному (есть строчные и прописные буквы). Необходимо првести всё к строчным буквам

In [149]:
data['education'] = data['education'].str.lower()
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Был использован метод astype,так как нам необходимо перевести значения столбцов из вещественного типа в целочисленный.
Тип данных столбцов days_employed и total_income был изменен с float на int. Первые пять строк таблицы показывают правильные целочисленные значения.

### Обработка дубликатов

In [150]:
data.duplicated().sum()

71

In [151]:
data = data.drop_duplicates().reset_index(drop=True)

In [152]:
data.duplicated().sum()

0

### Вывод

Для посика дубликатов был использован метод duplicated() с подсчётом количества дубликатов методом sum(). Для удаления дубликатов был использован метод drop_duplicates() с вызовом метода reset_index(drop=True) для изменения индексов и удаления старых. 
Возможными причинами появления дубликатов могут являтся повторные обращения человека в банк, а также технические особенности сбора данных.

### Лемматизация

In [153]:
data['purpose'].unique() #выводим список уникальных значений для стобца purpose

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [154]:
from pymystem3 import Mystem #подключаем библиотеку pymystem3
m = Mystem() 


### Вывод

Для лемматизации значений в столбце 'purpose' я воспользовался библиоткекой pymystem3.

### Категоризация данных

Видно, что основных целей пять: жильё, автомобиль, образование, свадьба, недвижимость. На мой взгляд, под понятием недвижимости могут быть коммерческие помещения и это не совсем то же, что и жильё. Соответственно, необходимо лемматизировать весь столбец для нахождения необходимых лемм, а затем категоризировать данные

In [155]:
import collections
lemma_join = ' '.join(data['purpose'])
lemma_purpose = m.lemmatize(lemma_join)
lemma_purpose_counter = collections.Counter(lemma_purpose).most_common(10)
print(lemma_purpose_counter)

[(' ', 55023), ('недвижимость', 6351), ('покупка', 5897), ('жилье', 4460), ('автомобиль', 4306), ('образование', 4013), ('с', 2918), ('операция', 2604), ('свадьба', 2324), ('свой', 2230)]


Применив метод most_common(х) мы получили 5 наиболее популярных цели :жильё, автомобиль, образование, свадьба, недвижимость

In [156]:
def lemmatize(purpose): #функция принимает значение всего столбца
    lemma = m.lemmatize(purpose)
    if 'жилье' in lemma:
        return 'жилье'
    if 'недвижимость' in lemma:
        return 'недвижимость'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'образование' in lemma:
        return 'образование'
data['purpose_lemm'] = data['purpose'].apply(lemmatize) #применяем функцию к столбцу purpose и создаем еще один столбец
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [157]:
data['purpose_lemm'].unique() #проверяем категоризацию

array(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'],
      dtype=object)

### Вывод

Данные в столбце 'purpose' лемматизированы и категоризированы. Столбец 'purpose_lemm' содержит всего 5 уникальных значений.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [158]:
data['children'].unique() #вызовим метод unique() для нахождения уникальных значений столбца children

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Обнаружено неправильно значение количества детей: -1. Необходимо посчитать сколько таких значений всего

In [159]:
data[data['children'] == -1]['children'].count()

47

Значений с отрицательным количеством детей :47. Это не большая сумма и, скорее всего это техническая ошибка. Имеет смысл перевести отрицательные значения в положительные 

In [160]:
data['children'] = abs(data['children']) #меняем отрицательные значения на положительные
data[data['children'] == -1]['children'].count() #проверяем

    

0

In [188]:
data['children'].median() #вычисляем медианное значениие количества детей


0.0

In [189]:
#Создаем функцию по замене значения 20 детей на медианное значение
def child_cut(children): 
    if children == 20:
        children = 0
    return children
data['children'] = data['children'].apply(child_cut)

In [190]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [191]:
def childless_count(debt): #создаём функцию для подсчёта количесвта без детей, с задолженностью/без задолженности
    child = data[(data['children']==0) & (data['debt']==debt)] 
    return child['children'].count()


In [192]:
def child_count(debt): #создаём функцию для подсчёта количесвта с детьми, с задолженностью/без задолженности
    child = data[(data['children']!=0) & (data['debt']==debt)] 
    return child['children'].count()

In [193]:
#Считаем общее количество значений без детей
childless_count_sum = childless_count(1) + childless_count(0)
#Считаем количество значений без детей и без задолженности по отношению к сумме
childless_without_debt = childless_count(0) / childless_count_sum
print(format(childless_without_debt, '.1%'))
#Считаем количество значений без детей и с задолженности по отношению к сумме
childless_with_debt = childless_count(1) / childless_count_sum
print(format(childless_with_debt, '.1%'))

92.4%
7.6%


In [194]:
#Считаем общее количество значений c детьми
child_count_sum = child_count(1) + child_count(0)
#Считаем количество значений c детьми и без задолженности по отношению к сумме
child_without_debt = child_count(0) / child_count_sum
print(format(child_without_debt, '.1%'))
#Считаем количество значений с детьми и с задолженности по отношению к сумме
child_with_debt = child_count(1) / child_count_sum
print(format(child_with_debt, '.1%'))

90.8%
9.2%


### Вывод

Количество бездетных и c задолженностью: 7,5% <br>
Количество с детьми и с задолженностью: 9,2% <br>
Разница составляет: 1,7%
 

Сгруппируем клиентов по количеству детей и посчитаем общее количество в каждой группе, а затем только количество должников (count() и sum() соответствено). Затем, разделив одно на другое получим искомый процент.

In [195]:
# Создадим сводную таблицу индексами будут количество детей, столбцы значений общее количество и должники
pivot = data.pivot_table(index='children', values= ['gender','debt'] ,aggfunc={'gender': 'count', 'debt':'sum'})
#Добавим столбец с процентным количеством должников
pivot['percent'] =  (pivot['debt'] / pivot['gender'])
#отформатируем для более наглядного представления
pivot.set_axis(['debt','total',  'percent'], axis='columns',inplace=True) 
def format_pivot(pivot): 
    return format(pivot, '.2%')
pivot['percent'] = pivot['percent'].apply(format_pivot)
pivot.sort_values(by='percent')

,debt,total,percent
children,,,
5,0,9,0.00%
0,1071,14167,7.56%
3,27,330,8.18%
1,445,4855,9.17%
2,194,2052,9.45%
4,4,41,9.76%


<h5> вывод после агрегации</h5>
Бездетные люди имеют меньше задолженностей (7,5% должников). Самое большое количество должников из тех, кто имеет 2 или 4 ребенка (~9,5% должников)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [196]:
data['family_status'].unique() #Найдём уникальные значения для столбца family_status


array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Из списка выше можно увидеть пять различных статусов. Создадим функцию, которая будет считать отношение значений без задолженности к общему количеству значений для каждого семейного статуса. Семейные статусы объединять в группы не имеет смысла, так как в реальности семейное положение у людей могут быть другие (например, люди женаты, но не живут вместе или не женаты, но , наоборот, живут) 

In [197]:
def family_status(fid): #создаём функцию для подсчёта зависимости семейного положения и задолженности
    #выделяем необходимые строки с задолженностью и без
    status_without_debt = data[(data['family_status_id']==fid) & (data['debt']==0)] 
    status_with_debt = data[(data['family_status_id']==fid) & (data['debt']==1)]
    #количество значений без задолженности
    status_without_debt = status_without_debt.count() 
    #количество значений с задолженностью
    status_with_debt = status_with_debt.count()
    #общее количество значений для данного статуса
    status_sum = status_without_debt + status_with_debt
    #соотношение количества значений без задолженности к общему количеству значений для данного статуса
    status_without_debt_total = status_without_debt / status_sum
    return status_without_debt_total['family_status_id']

In [198]:
#Применяем функцию ко всем пяти статусам и получаем необходимые данные
print(format(family_status(0), '.1%'))
print(format(family_status(1), '.1%'))
print(format(family_status(2), '.1%'))
print(format(family_status(3), '.1%'))
print(format(family_status(4), '.1%'))


92.5%
90.7%
93.4%
92.9%
90.2%


<h5> Решим задачу при помощи сводных таблиц</h5>


In [199]:
# Создадим сводную таблицу индексами будет статус, столбцы значений общее количество значений для данного статуса и должники
pivot_status = data.pivot_table(index='family_status', values= ['family_status_id','debt'] ,aggfunc={'family_status_id': 'count', 'debt':'sum'})
#Добавим столбец с процентным количеством должников
pivot_status['percent'] =  (pivot_status['debt'] / pivot_status['family_status_id'])
#отформатируем для более наглядного представления
pivot_status.set_axis(['debt','total', 'percent'], axis='columns',inplace=True) 
def format_pivot(pivot): 
    return format(pivot, '.2%')
pivot_status['percent'] = pivot_status['percent'].apply(format_pivot)
pivot_status.sort_values(by='percent')

,debt,total,percent
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12339,7.55%
гражданский брак,388,4151,9.35%
Не женат / не замужем,274,2810,9.75%


### Вывод

Наибольшее количество должников встречается среди неженатых(незамужних)людей и, людей, живущих в гражданском браке (~9,5%).<br>
Меньше всего должников - вдовцы (7,5%)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [200]:
data['total_income'].min()

20667

In [201]:
data['total_income'].max()

2265604

In [202]:
data['total_income'].median()

145017.0

Для подсчёта средней зарплаты мы используем медиану, так как она лучше подходит, в отличии от среднестатистического значения, для этих целей. Так как медиана равна 156 т.р., то имеет смысл разделить значения дохода на 2 категории: 
<ul>
    <li>до 156 т.р.</li>
    <li>больше 156 т.р.</li>
</ul>

In [181]:
def income_status_less(income): #создаём функцию для подсчёта зависимости дохода и задолженности
    #выделяем необходимые строки с задолженностью и без
    income_without_debt = data[(data['total_income'] < income) & (data['debt']==0)] 
    income_with_debt = data[(data['total_income'] < income) & (data['debt']==1)]
    #количество значений без задолженности
    income_without_debt = income_without_debt.count() 
    #количество значений с задолженностью
    income_with_debt = income_with_debt.count()
    #общее количество значений для данного дохода
    income_sum = income_without_debt + income_with_debt
    #соотношение количества значений без задолженности к общему количеству значений для данного доходаа
    income_without_debt_total = income_without_debt / income_sum
    return income_without_debt_total['total_income']

In [182]:
print(format(income_status_less(156000), '.1%'))

91.7%


In [183]:
def income_status_more(income): #создаём функцию для подсчёта зависимости дохода и задолженности
    #выделяем необходимые строки с задолженностью и без
    income_without_debt = data[(data['total_income'] > income) & (data['debt']==0)] 
    income_with_debt = data[(data['total_income'] > income) & (data['debt']==1)]
    #количество значений без задолженности
    income_without_debt = income_without_debt.count() 
    #количество значений с задолженностью
    income_with_debt = income_with_debt.count()
    #общее количество значений для данного дохода
    income_sum = income_without_debt + income_with_debt
    #соотношение количества значений без задолженности к общему количеству значений для данного дохода
    income_without_debt_total = income_without_debt / income_sum
    return income_without_debt_total['total_income']

In [184]:
print(format(income_status_more(156000), '.1%')) #

92.2%


### Вывод

Люди с доходом ниже 150 т.р. являются должниками в 8,3% случаев
Люди с доходом выше 150 т.р. являются должниками в 7,9% случаев
Соответственно, никакой зависимости между уровнем дохода и возвратом кредита в срок нет (0,4%). 

- Как разные цели кредита влияют на его возврат в срок?

In [185]:
def purpose(fid): #создаём функцию для подсчёта зависимости цели и задолженности
    #выделяем необходимые строки с задолженностью и без
    purpose_without_debt = data[(data['purpose_lemm']==fid) & (data['debt']==0)] 
    purpose_with_debt = data[(data['purpose_lemm']==fid) & (data['debt']==1)]
    #количество значений без задолженности
    purpose_without_debt = purpose_without_debt.count() 
    #количество значений с задолженностью
    purpose_with_debt = purpose_with_debt.count()
    #общее количество значений для данной цели
    purpose_sum = purpose_without_debt + purpose_with_debt
    #соотношение количества значений без задолженности к общему количеству значений для данной цели
    purpose_without_debt_total = purpose_without_debt / purpose_sum
    return purpose_without_debt_total['purpose_lemm']

In [186]:
# Применяем функцию к всем вариантам целей
print(format(purpose('жилье'), '.1%'))
print(format(purpose('недвижимость'), '.1%'))
print(format(purpose('автомобиль'), '.1%'))
print(format(purpose('свадьба'), '.1%'))
print(format(purpose('образование'), '.1%'))

93.1%
92.5%
90.6%
92.0%
90.8%


<h5> Решим задачу при помощи сводных таблиц</h5>

In [187]:
# Создадим сводную таблицу индексами будет цель, столбцы значений общее количество значений для данного статуса и должники
pivot_purpose = data.pivot_table(index='purpose_lemm', values= ['purpose','debt'] ,aggfunc={'purpose': 'count', 'debt':'sum'})
#Добавим столбец с процентным количеством должников
pivot_purpose['percent'] =  (pivot_purpose['debt'] / pivot_purpose['purpose'])
#отформатируем для более наглядного представления
pivot_purpose.set_axis(['debt','total', 'percent'], axis='columns',inplace=True) 
def format_pivot(pivot): 
    return format(pivot, '.2%')
pivot_purpose['percent'] = pivot_purpose['percent'].apply(format_pivot)
pivot_purpose.sort_values(by='percent')

,debt,total,percent
purpose_lemm,,,
жилье,308,4460,6.91%
недвижимость,474,6351,7.46%
свадьба,186,2324,8.00%
образование,370,4013,9.22%
автомобиль,403,4306,9.36%


### Вывод

Наименьшее количество должников у целей жильё (6.91%) и недвижимость (7.46%).<br>
Наибольшее количество должников у целей образование (9.22%) и автомобиль (9.36%)

### Шаг 4. Общий вывод

Наиболее подходящие категории для выдачи кредита это:
<ul>
 <li>количество детей: 0</li>
 <li>семейное положение: вдовец или в разводе</li>
 <li>цель кредита: жильё или недвижимость</li>
</ul>
Наименее подходящие категории для выдачи кредита это:
<ul>
 <li>количество детей: 2 или 4</li>
 <li>семейное положение: гражданский брак или не женат</li>
 <li>цель кредита: образование или автомобиль</li>
</ul>
Уровень дохода кандидата практически не влияет на наличие или отсутствие задолженности